<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# $p_r$, the radial component of the momentum vector, up to and including 3.5 post-Newtonian order

## This notebook constructs the radial component of the momentum vector, up to and including 3.5 PN order

**Notebook Status:** <font color='red'><b> IN PROGRESS </b></font>

**Validation Notes:** All expressions in this notebook were transcribed twice by hand on separate occasions, and expressions were corrected as needed to ensure consistency with published work. Published work was cross-validated and typo(s) in published work were corrected. In addition, this tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation). **Additional validation tests may have been performed, but are as yet, undocumented.**

## Author: Zach Etienne

### This notebook exists as the following Python module:
1. [PN_p_r.py](../../edit/NRPyPN/PN_p_r.py)

### This notebook & corresponding Python module depend on the following NRPy+/NRPyPN Python modules:
1. [outputC.py](../../edit/outputC.py): [**documentation+tutorial**](../Tutorial-Coutput__Parameter_Interface.ipynb)
1. [indexedexp.py](../../edit/indexedexp.py): [**documentation+tutorial**](../Tutorial-Indexed_Expressions.ipynb)
1. [NRPyPN_shortcuts.py](../../edit/NRPyPN/NRPyPN_shortcuts.py): [**documentation**](NRPyPN_shortcuts.ipynb)

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

1. Part 0: [Import needed Python modules](#imports)
1. Part 1: [Basic strategy](#strategy) for computing $p_r$
1. Part 2: [$p_r$](#p_r), up to and including 3.5PN order, as derived in [Ramos-Buades, Husa, and Pratten (2018)](https://arxiv.org/abs/1810.00036)
1. Part 2: [Validation against second transcription and corresponding Python module](#code_validation)
1. Part 3: [Validation against trusted numerical values](#code_validationv2) (i.e., in Table V of [Ramos-Buades, Husa, and Pratten (2018)](https://arxiv.org/abs/1810.00036))
1. Part 4: [LaTeX PDF output](#latex_pdf_output): $\LaTeX$ PDF Output

<a id='imports'></a>

# Part 0: Import needed Python modules \[Back to [top](#toc)\]
$$\label{imports}$$ 

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys                    # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
from outputC import *            # NRPy+: Core C code output module
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
from NRPyPN_shortcuts import *   # NRPyPN: shortcuts for e.g., vector operations

<a id='strategy'></a>

# Part 1: Basic strategy for computing $p_r$ \[Back to [top](#toc)\]
$$\label{strategy}$$ 


## Part 1.a: Construct the full Hamiltonian consistent with a binary orbiting instantaneously in the $x$-$y$ plane

As detailed in [Ramos-Buades, Husa, and Pratten (2018)](https://arxiv.org/abs/1810.00036), the basic strategy for computing $p_r$ first involves constructing the full Hamiltonian expression assuming that, consistent with a binary system initially on the $x$-axis and orbiting instantaneously on the $x$-$y$ plane, the momenta and normal vectors are given by:

\begin{align}
P_1^x &= -p_r\\
P_2^x &= +p_r\\
P_1^y &= +p_t\\
P_2^y &= -p_t\\
P_1^z = P_2^z &= 0\\
\mathbf{n} &= (1,0,0)
\end{align}

Now let's construct the full Hamiltonian, applying these assumptions as we go:

In [2]:
def f_Htot(m1,m2, n12U,n21U, S1U, S2U, p1U,p2U, q):
    def make_replacements(expr):
        zero = sp.sympify(0)
        one  = sp.sympify(1)
        return expr.subs(p1U[1],Pt).subs(p2U[1],-Pt).subs(p1U[2],zero).subs(p2U[2],zero).subs(p1U[0],-Pr).subs(p2U[0],Pr)\
    .subs(nU[0],one).subs(nU[1],zero).subs(nU[2],zero)\
    .subs(q,r)

    import PN_Hamiltonian_NS as H_NS
    H_NS.f_H_Newt__H_NS_1PN__H_NS_2PN(m1,m2, pU, nU, q)
    H_NS.f_H_NS_3PN(m1,m2, pU, nU, q)
    
    global Htot
    Htot = make_replacements(+H_NS.H_Newt
                             +H_NS.H_NS_1PN
                             +H_NS.H_NS_2PN
                             +H_NS.H_NS_3PN)

    import PN_Hamiltonian_SO as H_SO
    H_SO.f_H_SO_1p5PN(m1,m2, n12U,n21U, S1U, S2U, p1U,p2U, q)
    H_SO.f_H_SO_2p5PN(m1,m2, n12U,n21U, S1U, S2U, p1U,p2U, q)
    H_SO.f_H_SO_3p5PN(m1,m2, n12U,n21U, S1U, S2U, p1U,p2U, q)
    Htot+= make_replacements(+H_SO.H_SO_1p5PN
                             +H_SO.H_SO_2p5PN
                             +H_SO.H_SO_3p5PN)

    import PN_Hamiltonian_SS as H_SS
    H_SS.f_H_SS_2PN(m1,m2, S1U,S2U, nU, q)
    H_SS.f_H_SS_S1S2_3PN(m1,m2, n12U, S1U,S2U, p1U,p2U, q)
    H_SS.f_H_SS_S1sq_S2sq_3PN(m1,m2, n12U,n21U, S1U,S2U, p1U,p2U, q)
    Htot+= make_replacements(+H_SS.H_SS_2PN
                             +H_SS.H_SS_S1S2_3PN
                             +H_SS.H_SS_S1sq_S2sq_3PN)

    import PN_Hamiltonian_SSS as H_SSS
    H_SSS.f_H_SSS_3PN(m1,m2, n12U,n21U, S1U,S2U, p1U,p2U, q)
    Htot+= make_replacements(+H_SSS.H_SSS_3PN)
    
f_Htot(m1,m2, n12U,n21U, S1U, S2U, p1U,p2U, q)
print(sp.diff(Htot.subs(Pr,sp.sympify(0)),r))

9*Pt**2*S1U2*S2U2/(2*m2**2*r**4) + 9*Pt**2*S1U2*S2U2/(2*m1**2*r**4) + S1U2*(-2*(Pt**3/(m1*m2) + Pt*(3*Pt**2/(4*m1*m2) + 3*Pt**2/(4*m1**2) - 5*Pt**2*m2/(8*m1**3)))/r**3 - 3*(-Pt*(6*m1 + 15*m2/2) + Pt*(-11*m2/2 - 5*m2**2/m1))/r**4) + S1U2*(2*Pt*(Pt**4/(2*m1*m2**3) + Pt**4/(4*m1**2*m2**2) + Pt**4/(2*m1**3*m2))/r**3 - 2*Pt*(-3*Pt**4/(16*m1*m2**3) + 3*Pt**4/(16*m1**2*m2**2) - 3*Pt**4/(16*m1**3*m2) - 9*Pt**4/(16*m1**4) + 7*Pt**4*m2/(16*m1**5))/r**3 - 3*Pt*(-Pt**2*(23/(4*m1) + 9*m2/(2*m1**2)) - Pt**2*(37/(8*m2) + 159/(16*m1)) + Pt**2*(-3*m2/(2*m1**2) + 27*m2**2/(8*m1**3)))/r**4 + 3*Pt*(Pt**2*(5/m2 + 47/(8*m1)) + 53*Pt**2/(8*m2) + 13*Pt**2/(2*m1))/r**4 - 4*(Pt*(21*m1**2/2 + 473*m1*m2/16 + 63*m2**2/4) + Pt*(181*m1*m2/16 + 95*m2**2/4 + 75*m2**3/(8*m1)))/r**5) - 2*S1U2*(2*Pt + 3*Pt*m2/(2*m1))/r**3 + S2U2*(-2*(Pt**3/(m1*m2) + Pt*(-5*Pt**2*m1/(8*m2**3) + 3*Pt**2/(4*m2**2) + 3*Pt**2/(4*m1*m2)))/r**3 - 3*(-Pt*(15*m1/2 + 6*m2) + Pt*(-5*m1**2/m2 - 11*m1/2))/r**4) + S2U2*(2*Pt*(Pt**4/(2*m1*m2**3) + Pt**

## Computing $p_r$ as a function of $\frac{dr}{dt}$

Hamilton's equations of motion imply that
$$
\frac{dr}{dt} = \frac{\partial H}{\partial p_r}.
$$

Next we Taylor expand $\frac{\partial H}{\partial p_r}$ in powers of $p_r$, about $p_r=0$:

\begin{align}
\frac{dr}{dt} = \frac{\partial H}{\partial p_r} = \left.\frac{\partial H}{\partial p_r}\right|_{p_r=0} 
+ \frac{1}{1!} p_r \left.\frac{\partial^2 H}{\partial p_r^2}\right|_{p_r=0} 
+ \frac{1}{2!} p_r^2 \left.\frac{\partial^3 H}{\partial p_r^3}\right|_{p_r=0} 
+ \frac{1}{3!} p_r^3 \left.\frac{\partial^4 H}{\partial p_r^4}\right|_{p_r=0} 
+ \mathcal{O}(p_r^4)
\end{align}
so to first order we get

$$
p_r \approx \left(\frac{dr}{dt} - \left.\frac{\partial H}{\partial p_r}\right|_{p_r=0} \right) \left( \left.\frac{\partial^2 H}{\partial p_r^2}\right|_{p_r=0} \right)^{-1}.
$$

Given the input masses and spins, we can compute $p_t$ using the formula given in [this NRPyPN notebook](PN-p_t.ipynb) (from [Ramos-Buades, Husa, and Pratten (2018)](https://arxiv.org/abs/1810.00036)), and the above will lead us with one equation and two unknowns: $p_r$ and $\frac{dr}{dt}$. This is an equation we can derive directly from our Hamiltonian expression, and compare the output to the expression derived to 3.5PN order in [Ramos-Buades, Husa, and Pratten (2018)](https://arxiv.org/abs/1810.00036).

In [3]:
# Next we compute p_r as a function of dr_dt (unknown) and known quantities using 
# p_r ~ (\frac{dr}{dt} - partial_H/partial_{p_r}|_{p_r=0} / \partial^2_H/partial_{p_r^2}|_{p_r=0}
def f_p_r(H,drdt):
    dHdpr_przero   = sp.diff(H,Pr).subs(Pr,sp.sympify(0))
    d2Hdpr2_przero = sp.diff(sp.diff(H,Pr),Pr).subs(Pr,sp.sympify(0))
    global p_r
    p_r = (drdt - dHdpr_przero)/(d2Hdpr2_przero)

## Computing $\frac{dr}{dt}$

Now we have $p_r$ as a function of $\frac{dr}{dt}$ (unknown) and known input parameters (like masses and spins). Let's next construct an expression for $\frac{dr}{dt}$ in terms of known quantities, via
$$
\frac{dr}{dt}=\left(\frac{dE_{\rm GW}}{dt}+\frac{dM}{dt}\right)\left[\frac{dH_{\rm circ}}{dr}\right]^{-1},
$$
where
$$
\frac{dH_{\rm circ}(r,p_t(r))}{dr} = \frac{\partial H(p_r=0)}{\partial r} 
+ \frac{\partial H(p_r=0)}{\partial p_t} \frac{\partial p_t}{\partial r},
$$
and the total energy flux
$$
\mathcal{E}(M\Omega,...) = \left(\frac{dE_{\rm GW}}{dt}+\frac{dM}{dt}\right) 
$$ 
is given in terms of input parameters (e.g., masses and spins), plus $M\Omega$, which we also constructed in terms of the input masses, spins, and orbital separation $r$.

In [4]:
def f_dr_dt(Htot, m1,m2, n12U,n21U, chi1U,chi2U, S1U,S2U, p1U,p2U, r):
    # First compute M\Omega
    import PN_MOmega as MOm
    MOm.f_MOmega(m1,m2, chi1U,chi2U, r)

    # Then compute dE_GW_dt_plus_dM_dt
    import PN_dE_GW_dt_and_dM_dt as dEdt
    dEdt.f_dE_GW_dt_and_dM_dt(MOm.MOmega, m1,m2, n12U, S1U,S2U)

    # Then compute p_t
    import PN_p_t as pt
    pt.f_p_t(m1,m2, chi1U,chi2U, r)

    # Then compute dH_{circ}/dr
    dHcirc_dr = (+sp.diff(Htot.subs(Pr,sp.sympify(0)),r) 
                 +sp.diff(Htot.subs(Pr,sp.sympify(0)),Pt)*sp.diff(pt.p_t,r))

    global dr_dt
    dr_dt = dEdt.dE_GW_dt_plus_dM_dt / dHcirc_dr

<a id='code_validation'></a>

# Part 2: Validation of transcribed versions for $p_r$ \[Back to [top](#toc)\]
$$\label{code_validation}$$ 

As a code validation check, we verify agreement between 
* the SymPy expressions transcribed from the cited published work on two separate occasions, and

Next, for validation purposes we will include the expression for $p_r$ given by Eq 2.16 of [Ramos-Buades, Husa, and Pratten (2018)](https://arxiv.org/abs/1810.00036). 

To reduce the possibility of copying error, the equation for $p_r$ is taken directly from the arXiv LaTeX source code of Eq 2.16 in [Ramos-Buades, Husa, and Pratten (2018)](https://arxiv.org/abs/1810.00036), and only mildly formatted to (1) improve presentation in Jupyter notebooks and (2) to ensure some degree of consistency in notation across different terms in other NRPyPN notebooks:

\begin{equation}
\begin{split}
p_r &= \left[-\frac{dr}{dt} \right.\\
&\quad\quad \left. +\frac{1}{r^{7/2}} \left( -\frac{(6 q+13) q^2 \chi _{1x} \chi _{2 y}}{4 (q+1)^4}-\frac{(6 q+1) q^2 \chi _{2 x} \chi _{2 y}}{4 (q+1)^4}+\chi _{1y} \left(-\frac{q (q+6) \chi _{1x}}{4 (q+1)^4}-\frac{q (13 q+6) \chi _{2 x}}{4 (q+1)^4}\right)\right) \right.\\
&\quad\quad \left. +\frac{1}{r^4} \left( \chi _{1z} \left(\frac{3 q (5 q+2) \chi _{1x} \chi _{2 y}}{2 (q+1)^4}   -\frac{3 q^2 (2 q+5) \chi _{2 x} \chi _{2 y}}{2 (q+1)^4}\right)+\chi _{1y} \chi _{2 z} \left(\frac{3 q^2 (2 q+5) \chi _{2 x}}{2 (q+1)^4}-\frac{3 q (5 q+2) \chi _{1x}}{2 (q+1)^4}\right)\right)\right] \\
& \times \left[ -\frac{(q+1)^2}{q}-\frac{1 \left(-7 q^2-15 q-7\right)}{2 q r}  \right.
 \\
&\quad\quad \left.  -\frac{47 q^4+229 q^3+363 q^2+229 q+47}{8 q (q+1)^2 r^2} -\frac{1}{r^{5/2}}\left( \frac{\left(4 q^2+11 q+12\right) \chi _{1z}}{4 q (q+1)}+\frac{\left(12 q^2+11 q+4\right) \chi _{2 z}}{4 (q+1)} \right)    \right. \\
&\quad\quad \left. \left.- \frac{1}{r^{7/2}}  \left( \frac{\left(-53 q^5-357 q^4-1097 q^3-1486 q^2-842 q-144\right) \chi _{1z}}{16 q (q+1)^4}+\frac{\left(-144 q^5-842 q^4-1486 q^3-1097 q^2-357 q-53\right) \chi _{2 z}}{16 (q+1)^4} \right)\right. \right. \\
&\quad\quad \left. -\frac{1}{r^3} \left(\frac{\left(q^2+9 q+9\right) \chi _{1x}^2}{2 q (q+1)^2}+\frac{\left(3 q^2+5 q+3\right) \chi _{2 x} \chi _{1x}}{(q+1)^2}+\frac{\left(3 q^2+8 q+3\right) \chi _{1y} \chi _{2 y}}{2 (q+1)^2}-\frac{9 q^2 \chi _{2 y}^2}{4 (q+1)}+\frac{\left(3 q^2+8 q+3\right) \chi _{1z} \chi _{2 z}}{2 (q+1)^2}-\frac{9 q^2 \chi _{2 z}^2}{4 (q+1)} \right. \right.
\\
&\quad\quad\quad \left. \left. +\frac{\left(9 q^3+9 q^2+q\right) \chi _{2 x}^2}{2 (q+1)^2}+\frac{-363 q^6-2608 q^5-7324 q^4-10161 q^3-7324 q^2-2608 q-363}{48 q (q+1)^4}-\frac{9 \chi _{1y}^2}{4 q (q+1)}-\frac{9 \chi _{1z}^2}{4 q (q+1)}-\frac{\pi ^2}{16} \right) \right]^{-1}. 
\end{split}
\end{equation}

In [5]:
# Transcribed from Eq 2.18 of 
# Ramos-Buades, Husa, and Pratten (2018),
#   https://arxiv.org/abs/1810.00036
def f_p_r_RHP2018(drdt, m1,m2, chi1x,chi1y,chi1z, chi2x,chi2y,chi2z,r):
    q = m2/m1 # It is assumed that q >= 1, so m2 >= m1.
    p_r_num = (-drdt
               +(-(6*q+13)*q**2*chi1x*chi2y/(4*(q+1)**4)
                 -(6*q+ 1)*q**2*chi2x*chi2y/(4*(q+1)**4)
                 +chi1y*(-q*(   q+6)*chi1x/(4*(q+1)**4)
                         -q*(13*q+6)*chi2x/(4*(q+1)**4)))/r**div(7,2)
               +(+chi1z*(+3*q   *(5*q+2)*chi1x*chi2y/(2*(q+1)**4)
                         -3*q**2*(2*q+5)*chi2x*chi2y/(2*(q+1)**4))
                 +chi1y*chi2z*(+3*q**2*(2*q+5)*chi2x/(2*(q+1)**4)
                               -3*q   *(5*q+2)*chi1x/(2*(q+1)**4)))/r**4)
    p_r_den = (-(q+1)**2/q - (-7*q**2-15*q-7)/(2*q*r)
               -(47*q**4 + 229*q**3 + 363*q**2 + 229*q + 47)/(8*q*(q+1)**2*r**2)
               -(+( 4*q**2 + 11*q + 12)*chi1z/(4*q*(q+1))
                 +(12*q**2 + 11*q +  4)*chi2z/(4*  (q+1)))/r**div(5,2)
               -(+(- 53*q**5 - 357*q**4 - 1097*q**3 - 1486*q**2 - 842*q - 144)*chi1z/(16*q*(q+1)**4)
                 +(-144*q**5 - 842*q**4 - 1486*q**3 - 1097*q**2 - 357*q -  53)*chi2z/(16  *(q+1)**4))/r**div(7,2)
               -(+(  q**2 + 9*q + 9)*chi1x**2/(2*q*(q+1)**2)
                 +(3*q**2 + 5*q + 3)*chi2x*chi1x/((q+1)**2)
                 +(3*q**2 + 8*q + 3)*chi1y*chi2y/(2*(q+1)**2)
                 -9*q**2*chi2y**2/(4*(q+1))
                 +(3*q**2 + 8*q + 3)*chi1z*chi2z/(2*(q+1)**2)
                 -9*q**2*chi2z**2/(4*(q+1))
                 +(9*q**3 + 9*q**2 + q)*chi2x**2/(2*(q+1)**2)
                 +(-363*q**6 - 2608*q**5 - 7324*q**4 - 10161*q**3 - 7324*q**2 - 2608*q - 363)/(48*q*(q+1)**4)
                 -9*chi1y**2/(4*q*(q+1))
                 -9*chi1z**2/(4*q*(q+1)) - sp.pi**2/16)/r**3)
    global p_r_RHP2018
    p_r_RHP2018 = p_r_num/p_r_den

In [6]:
# Second version, used for validation purposes only.
def f_p_r_RHP2018v2(drdt, m1,m2, chi1x,chi1y,chi1z, chi2x,chi2y,chi2z, r):
    q = m2/m1
    p_r_num = (-drdt
               +(-(6*q+13)*q**2*chi1x*chi2y/(4*(q+1)**4) 
                 -(6*q+1)*q**2*chi2x*chi2y/(4*(q+1)**4)
                 +chi1y*(-q*(q+6)*chi1x/(4*(q+1)**4) - q*(13*q+6)*chi2x/(4*(q+1)**4)))/r**div(7,2)
               +(+chi1z*(+div(3,2)*(q*   (5*q+2)*chi1x*chi2y)/(q+1)**4
                         -div(3,2)*(q**2*(2*q+5)*chi2x*chi2y)/(q+1)**4)
                 +chi1y*chi2z*(+div(3,2)*(q**2*(2*q+5)*chi2x)/(q+1)**4
                               -div(3,2)*(q*   (5*q+2)*chi1x)/(q+1)**4))/r**4)
    p_r_den = (-(q+1)**2/q 
               -(-7*q**2-15*q-7)/(2*q*r)
               -(47*q**4 + 229*q**3 + 363*q**2 + 229*q + 47)/(8*q*(q+1)**2*r**2)
               -( (4*q**2+11*q+12)*chi1z/(4*q*(q+1)) + (12*q**2+11*q+4)*chi2z/(4*(q+1)) )/r**div(5,2)
               -(+(-53 *q**5 - 357*q**4 - 1097*q**3 - 1486*q**2 - 842*q - 144)*chi1z/(16*q*(q+1)**4)
                 +(-144*q**5 - 842*q**4 - 1486*q**3 - 1097*q**2 - 357*q - 53 )*chi2z/(16*  (q+1)**4) )/r**div(7,2)
               -(+(  q**2+9*q+9)*chi1x**2/(2*q*(q+1)**2)
                 +(3*q**2+5*q+3)*chi2x*chi1x/((q+1)**2)
                 +(3*q**2+8*q+3)*chi1y*chi2y/(2*(q+1)**2)
                 -(9*q**2*chi2y**2/(4*(q+1)))
                 +(3*q**2+8*q+3)*chi1z*chi2z/(2*(q+1)**2)
                 -(9*q**2*chi2z**2/(4*(q+1)))
                 +(9*q**3+9*q**2+q)*chi2x**2/(2*(q+1)**2)
                 +(-363*q**6 - 2608*q**5 - 7324*q**4 - 10161*q**3 - 7324*q**2 - 2608*q - 363)/(48*q*(q+1)**4)
                 -(9*chi1y**2)/(4*q*(q+1))
                 -(9*chi1z**2)/(4*q*(q+1))
                 -sp.pi**2/16) / r**3)
    global p_r_RHP2018v2
    p_r_RHP2018v2 = p_r_num/p_r_den

In [7]:
from outputC import *   # NRPy+: Core C code output module

def error(varname):
    print("ERROR: When comparing Python module & notebook, "+varname+" was found not to match.")
    sys.exit(1)

# Validation against second transcription of the expressions:
f_p_r_RHP2018(  drdt, m1,m2, chi1U[0],chi1U[1],chi1U[2], chi2U[0],chi2U[1],chi2U[2], q)
f_p_r_RHP2018v2(drdt, m1,m2, chi1U[0],chi1U[1],chi1U[2], chi2U[0],chi2U[1],chi2U[2], q)

if sp.simplify(p_r_RHP2018 - p_r_RHP2018v2) != 0: error("p_r_RHP2018v2")
print("TRANSCRIPTION TEST PASSES")

TRANSCRIPTION TEST PASSES


<a id='code_validationpt2'></a>

## Part 2.a: Numerical validation of transcribed version against $p_r$ expression given from Hamiltonian \[Back to [top](#toc)\]
$$\label{code_validationpt2}$$ 

As an additional validation check, we verify agreement between 

* the SymPy expressions generated in this notebook, and the corresponding Python module.

In [8]:
import random

def eval_random(errorsum,i,trusted, other, p_t):
    random.seed(i)

    qmassratio = 1.0 + 7*random.random()  # must be >= 1
    nr         = 10. + 3*random.random()  # Orbital separation
    # Choose spins so that the total spin magnitude does not exceed 1.
    nchi1x     = -0.55 + 1.1*random.random()
    nchi1y     = -0.55 + 1.1*random.random()
    nchi1z     = -0.55 + 1.1*random.random()
    nchi2x     = -0.55 + 1.1*random.random()
    nchi2y     = -0.55 + 1.1*random.random()
    nchi2z     = -0.55 + 1.1*random.random()

    nPt        = num_eval(p_t,
                          qmassratio=qmassratio, nr=nr,
                          nchi1x=nchi1x,nchi1y=nchi1y,nchi1z=nchi1z,
                          nchi2x=nchi2x,nchi2y=nchi2y,nchi2z=nchi2z)
    ndrdt      = 0.1*nPt*(random.random()+0.1)

    trusted_res = num_eval(trusted,
                           qmassratio=qmassratio, nr=nr,
                           nchi1x=nchi1x,nchi1y=nchi1y,nchi1z=nchi1z,
                           nchi2x=nchi2x,nchi2y=nchi2y,nchi2z=nchi2z,
                           ndrdt=ndrdt,nPt=nPt)
    other_result= num_eval(other,
                           qmassratio=qmassratio, nr=nr,
                           nchi1x=nchi1x,nchi1y=nchi1y,nchi1z=nchi1z,
                           nchi2x=nchi2x,nchi2y=nchi2y,nchi2z=nchi2z,
                           ndrdt=ndrdt,nPt=nPt)

    relerror = abs((trusted_res-other_result)/trusted_res)
    print("%.3e" % ((relerror+errorsum)/(i+1)), i+1, #"%.2f" % int(count+1)/int(i+1), 
          "%.4e" % relerror,"%.4e" % trusted_res,"%.4e" % other_result,
          "%.2f" % qmassratio,"%.2f" % nr,
          "%.2f" % nchi1x,"%.2f" % nchi1y,"%.2f" % nchi1z,
          "%.2f" % nchi2x,"%.2f" % nchi2y,"%.2f" % nchi2z)
    return relerror

import PN_p_t as pt
pt.f_p_t(m1,m2, chi1U,chi2U, q)

f_p_r(Htot,drdt)
f_p_r_RHP2018(  drdt, m1,m2, chi1U[0],chi1U[1],chi1U[2], chi2U[0],chi2U[1],chi2U[2], q)

errorsum = 0.0
for i in range(3):
    errorsum += eval_random(errorsum,i,p_r_RHP2018,p_r,pt.p_t)
# Without 3.5PN term in p_r denominator (comparing 
#      Eq 2.16 of https://arxiv.org/pdf/1810.00036.pdf with
#      Eq 16 of https://arxiv.org/pdf/1702.00872.pdf): 
#  average relative error over 1000 inputs = 4.112e-03
# 4.112e-03 1000 1.3856e-04 6.5871e-04 6.5861e-04 6.47 10.24 0.41 0.08 -0.01 -0.40 0.33 -0.20

# With 3.5PN term in p_r: average relative error over 1000 inputs = 4.027e-03
# 4.027e-03 1000 5.4787e-04 6.5825e-04 6.5861e-04 6.47 10.24 0.41 0.08 -0.01 -0.40 0.33 -0.20

1.669e-03 1 1.6691e-03 3.2027e-04 3.2081e-04 6.91 12.27 -0.09 -0.27 0.01 -0.10 0.31 -0.22
2.030e-03 2 2.3909e-03 4.2484e-04 4.2382e-04 1.94 12.54 0.29 -0.27 -0.01 -0.06 0.17 0.32
2.012e-03 3 1.9747e-03 3.1814e-04 3.1877e-04 7.69 12.84 -0.49 -0.46 0.37 0.26 0.19 -0.21


<a id='code_validationpub'></a>
# Part 4: Validation against trusted numerical values (i.e., in Table V of [Ramos-Buades, Husa, and Pratten (2018)](https://arxiv.org/abs/1810.00036)) \[Back to [top](#toc)\]

$$\label{code_validationpub}$$ 

We will measure success in this validation to be within the error bar of the two values of $p_r$ given for each iteration, preferably being closer to the second iteration's value than the first.

In [9]:
# First construct symbolic expression for p_r
f_dr_dt(Htot, m1,m2, n12U,n21U, chi1U,chi2U, S1U,S2U, p1U,p2U, r) # <- Note that we must pass r to the function here
#                                                                      as q has been replaced by r by SymPy in Htot. 
f_p_r(Htot,drdt)

In [30]:
# Useful function for comparing published & NRPyPN results
def compare_pub_NPN(desc,pub0,pub1,  qmassratio, nr, nchi1x,nchi1y,nchi1z, nchi2x,nchi2y,nchi2z):
    nPt        = num_eval(pt.p_t,
                          qmassratio=qmassratio, nr=nr,
                          nchi1x=nchi1x,nchi1y=nchi1y,nchi1z=nchi1z,
                          nchi2x=nchi2x,nchi2y=nchi2y,nchi2z=nchi2z)
    ndrdt      = num_eval(dr_dt,
                          qmassratio=qmassratio, nr=nr,
                          nchi1x=nchi1x,nchi1y=nchi1y,nchi1z=nchi1z,
                          nchi2x=nchi2x,nchi2y=nchi2y,nchi2z=nchi2z,
                          nPt=nPt).subs(gamma_EulerMascheroni,0.5772156649015328606065120900824024310421)

    NPN_result = num_eval(p_r,  qmassratio = qmassratio, nr=nr,
                          nchi1x=nchi1x, nchi1y=nchi1y, nchi1z=nchi1z,
                          nchi2x=nchi2x, nchi2y=nchi2y, nchi2z=nchi2z,
                          nPt=nPt,  ndrdt=ndrdt)
    
    print("##################################################")
    print("  "+desc)
    print("##################################################")
    print(str(pub0) + "          <- Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018)")
    print(str(NPN_result) + " <- Result from NRPyPN")
    print(str(pub1) + "          <- Result at 2nd iteration, from Table V of Ramos-Buades, Husa, and Pratten (2018)")
    relerror = abs((pub0-NPN_result)/pub0)
    relerror01 = abs((pub0-pub1)/pub1)
    print("Relative error between published iteration 0 vs it. 1: "+str(relerror01*100)+"%")
    resultstring = "Relative error between NRPyPN & published: "+str(relerror*100)+"%"
    if relerror > relerror01:
        resultstring += " < "+str(relerror01*100)+"%  <--- NOT GOOD!"
    else:
        resultstring += " < "+str(relerror01*100)+"%  <--- EXCELLENT AGREEMENT!"
    print(resultstring+"\n")

In [31]:
# 1. Let's consider the case:
# * Mass ratio q=1, chi1=chi2=(0,0,0), radial separation r=12
pub_result0 = 0.53833e-3 # Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018) https://arxiv.org/abs/1810.00036
pub_result1 = 0.468113e-3 # Expected result 2nd iteration, from Table V of Ramos-Buades, Husa, and Pratten (2018) https://arxiv.org/abs/1810.00036
qmassratio = 1.0  # must be >= 1
nr         = 12.  # Orbital separation
# Choose spins so that the total spin magnitude does not exceed 1.
nchi1x     = +0.
nchi1y     = +0.
nchi1z     = +0.
nchi2x     = +0.
nchi2y     = +0.
nchi2z     = +0.

compare_pub_NPN("Case: q=1, nonspinning, initial separation 12",pub_result0,pub_result1,
                qmassratio, nr, nchi1x,nchi1y,nchi1z, nchi2x,nchi2y,nchi2z)

##################################################
  Case: q=1, nonspinning, initial separation 12
##################################################
0.00053833          <- Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018)
0.000539170939050403 <- Result from NRPyPN
0.000468113          <- Result at 2nd iteration, from Table V of Ramos-Buades, Husa, and Pratten (2018)
Relative error between published iteration 0 vs it. 1: 15.000010681181674%
Relative error between NRPyPN & published: 0.156212555570496% < 15.000010681181674%  <--- EXCELLENT AGREEMENT!



In [32]:
# 2. Let's consider the case:
# * Mass ratio q=1.5, chi1= (0,0,-0.6); chi2=(0,0,0.6), radial separation r=10.8
pub_result0 = 0.699185e-3 # Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018) https://arxiv.org/abs/1810.00036
pub_result1 = 0.641051e-3 # Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018) https://arxiv.org/abs/1810.00036
qmassratio =  1.5  # must be >= 1
nr         = 10.8  # Orbital separation
nchi1x     = +0.
nchi1y     = +0.
nchi1z     = -0.6
nchi2x     = +0.
nchi2y     = +0.
nchi2z     = +0.6
compare_pub_NPN("Case: q=1.5, chi1z=-0.6, chi2z=0.6, initial separation 10.8",pub_result0,pub_result1,
                qmassratio, nr, nchi1x,nchi1y,nchi1z, nchi2x,nchi2y,nchi2z)

##################################################
  Case: q=1.5, chi1z=-0.6, chi2z=0.6, initial separation 10.8
##################################################
0.000699185          <- Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018)
0.000675943948454396 <- Result from NRPyPN
0.000641051          <- Result at 2nd iteration, from Table V of Ramos-Buades, Husa, and Pratten (2018)
Relative error between published iteration 0 vs it. 1: 9.068545248349965%
Relative error between NRPyPN & published: 3.32402033018504% < 9.068545248349965%  <--- EXCELLENT AGREEMENT!



In [33]:
# 3. Let's consider the case:
# * Mass ratio q=4, chi1= (0,0,-0.8); chi2=(0,0,0.8), radial separation r=11
pub_result0 = 0.336564e-3 # Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018) https://arxiv.org/abs/1810.00036
pub_result1 = 0.24708e-3 # Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018) https://arxiv.org/abs/1810.00036
qmassratio =  4.0  # must be >= 1
nr         = 11.0  # Orbital separation
nchi1x     = +0.
nchi1y     = +0.
nchi1z     = -0.8
nchi2x     = +0.
nchi2y     = +0.
nchi2z     = +0.8
compare_pub_NPN("Case: q=4.0, chi1z=-0.8, chi2z=0.8, initial separation 11.0",pub_result0,pub_result1,
                qmassratio, nr, nchi1x,nchi1y,nchi1z, nchi2x,nchi2y,nchi2z)

##################################################
  Case: q=4.0, chi1z=-0.8, chi2z=0.8, initial separation 11.0
##################################################
0.000336564          <- Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018)
0.000251239965029368 <- Result from NRPyPN
0.00024708          <- Result at 2nd iteration, from Table V of Ramos-Buades, Husa, and Pratten (2018)
Relative error between published iteration 0 vs it. 1: 36.21661000485673%
Relative error between NRPyPN & published: 25.3515037171628% < 36.21661000485673%  <--- EXCELLENT AGREEMENT!



In [34]:
# 4. Let's consider the case:
# * Mass ratio q=2, chi1= (0,0,0); chi2=(−0.3535, 0.3535, 0.5), radial separation r=10.8
pub_result0 = 0.531374e-3 # Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018) https://arxiv.org/abs/1810.00036
pub_result1 = 0.448148e-3 # Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018) https://arxiv.org/abs/1810.00036
qmassratio =  2.0  # must be >= 1
nr         = 10.8  # Orbital separation
nchi1x     = +0.
nchi1y     = +0.
nchi1z     = +0.
nchi2x     = -0.3535
nchi2y     = +0.3535
nchi2z     = +0.5
compare_pub_NPN("Case: q=2.0, chi2x=-0.3535, chi2y=+0.3535, chi2z=+0.5, initial separation 10.8",pub_result0,pub_result1,
                qmassratio, nr, nchi1x,nchi1y,nchi1z, nchi2x,nchi2y,nchi2z)

##################################################
  Case: q=2.0, chi2x=-0.3535, chi2y=+0.3535, chi2z=+0.5, initial separation 10.8
##################################################
0.000531374          <- Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018)
0.000545356534220444 <- Result from NRPyPN
0.000448148          <- Result at 2nd iteration, from Table V of Ramos-Buades, Husa, and Pratten (2018)
Relative error between published iteration 0 vs it. 1: 18.571097048296554%
Relative error between NRPyPN & published: 2.63139224358813% < 18.571097048296554%  <--- EXCELLENT AGREEMENT!



In [35]:
# 5. Let's consider the case:
# * Mass ratio q=8, chi1= (0, 0, 0.5); chi2=(0, 0, 0.5), radial separation r=11
pub_result0 = 0.102969e-3 # Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018) https://arxiv.org/abs/1810.00036
pub_result1 = 0.139132e-3 # Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018) https://arxiv.org/abs/1810.00036
qmassratio =  8.0  # must be >= 1
nr         = 11.0  # Orbital separation
nchi1x     = +0.
nchi1y     = +0.
nchi1z     = +0.5
nchi2x     = +0.
nchi2y     = +0.
nchi2z     = +0.5
compare_pub_NPN("""
Case: q=8.0, chi1z=chi2z=+0.5, initial separation 11

Note: This one is weird. Clearly the value in the table
      has a typo, such that the p_r and p_t values
      should be interchanged; p_t is about 20% the 
      next smallest value in the table, and the 
      parameters aren't that different. We therefore
      assume that this is the case, and find agreement
      with the published result to about 0.07%, which
      isn't the best, but given that the table values
      seem to be clearly wrong, it's an encouraging 
      sign.
""",pub_result0,pub_result1,
                qmassratio, nr, nchi1x,nchi1y,nchi1z, nchi2x,nchi2y,nchi2z)

##################################################
  
Case: q=8.0, chi1z=chi2z=+0.5, initial separation 11

Note: This one is weird. Clearly the value in the table
      has a typo, such that the p_r and p_t values
      should be interchanged; p_t is about 20% the 
      next smallest value in the table, and the 
      parameters aren't that different. We therefore
      assume that this is the case, and find agreement
      with the published result to about 0.07%, which
      isn't the best, but given that the table values
      seem to be clearly wrong, it's an encouraging 
      sign.

##################################################
0.000102969          <- Expected result, from Table V of Ramos-Buades, Husa, and Pratten (2018)
9.76943754276420e-5 <- Result from NRPyPN
0.000139132          <- Result at 2nd iteration, from Table V of Ramos-Buades, Husa, and Pratten (2018)
Relative error between published iteration 0 vs it. 1: 25.991863841531792%
Relative error between NRPyPN & pu

<a id='code_validationpython'></a>

# Part 5: Validation against corresponding Python module \[Back to [top](#toc)\]
$$\label{code_validationpython}$$ 

Here we verify agreement between 

* the SymPy expressions generated in this notebook, and the corresponding Python module.

<a id='latex_pdf_output'></a>

# Part 5: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[PN-p_r.pdf](PN-p_r.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [12]:
import os,sys                    # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("PN-p_r",location_of_template_file=os.path.join(".."))

Created PN-p_r.tex, and compiled LaTeX file to PDF file PN-p_r.pdf
